# Comptage des mots d'un corpus

In [1]:
cd ..

/Users/fm/Desktop/Work/Patat


In [2]:
import pandas as pd

## Chargement Corpus

In [5]:
corpus_filename = '220930-RecueilComplet.csv'

In [7]:
df_corpus = pd.read_csv(f'data/corpus/{corpus_filename}')

In [10]:
df_corpus = df_corpus[df_corpus['parse_method']=='parse']

In [11]:
df_corpus

,url,title,article,parse_method,date,tag,abstract,author,date_iso,words,...,des pages et des pages,des chiffres et des tableaux,sophisme,inversion de la preuve,invérifiable,propos rapporté,sources citées,titre décalé,site,text
2,https://www.breizh-info.com/2022/02/23/180457/...,Tests – vaccins – pass : fabrique de la paniqu...,Cela fait 2 ans que la crise du coronavirus a ...,parse,Le \n\t\t\t\t\t\t\t\t\t\t23 février 2022,"['Santé', 'covid', 'crise', ""décoder l'éco"", '...",NaN,NaN,2022-02-23T00:00:00,NaN,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,www.breizh-info.com,Tests – vaccins – pass : fabrique de la paniqu...
3,https://www.breizh-info.com/2022/02/20/180182/...,"Covid-19. La pandémie et les vaccins, pour mie...","Ces dernières semaines, le monde entier a prêt...",parse,Le \n\t\t\t\t\t\t\t\t\t\t20 février 2022,"['A La Une', 'Santé', ""coup d'etat mondial"", '...",NaN,NaN,2022-02-20T00:00:00,NaN,...,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,www.breizh-info.com,"Covid-19. La pandémie et les vaccins, pour mie..."
4,https://www.breizh-info.com/2022/04/19/185260/...,Covid-19. Une épidémie quasiment terminée avan...,Voici mon avant dernier bulletin de situation ...,parse,Le \n\t\t\t\t\t\t\t\t\t\t19 avril 2022,"['A La Une', 'Santé', 'COVID-19', 'epidemie', ...",NaN,NaN,2022-04-19T00:00:00,NaN,...,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,www.breizh-info.com,Covid-19. Une épidémie quasiment terminée avan...
5,https://www.breizh-info.com/2022/04/15/184984/...,"Piscine, voile, ramadan : scènes de la vie ord...",On a beaucoup parlé du Grand remplacement dura...,parse,Le \n\t\t\t\t\t\t\t\t\t\t15 avril 2022,"['Education', 'Société', 'burkini', 'hallal', ...",NaN,NaN,2022-04-15T00:00:00,NaN,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,www.breizh-info.com,"Piscine, voile, ramadan : scènes de la vie ord..."
6,https://www.breizh-info.com/2021/11/02/173738/...,Faut-il vous métisser comme le suggère la pres...,Comme nous le verrons dans l’ensemble des étud...,parse,Le \n\t\t\t\t\t\t\t\t\t\t2 novembre 2021,"['Sciences', 'métis', 'metissage', 'race']",NaN,NaN,2021-11-02T00:00:00,NaN,...,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,www.breizh-info.com,Faut-il vous métisser comme le suggère la pres...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
463,https://www.wikistrike.com/2022/08/fresne-cett...,Fresnes: cette prison où les condamnés finisse...,Cette photo est une erreur \n\nEt nul n'est au...,parse,Publié par wikistrike.com\n ...,[],NaN,NaN,2022-08-21T00:00:00,NaN,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,www.wikistrike.com,Fresnes: cette prison où les condamnés finisse...
464,https://www.wikistrike.com/2022/08/le-forum-ec...,Le Forum économique mondial estime qu’il exist...,Ne trahissant pas sa réputation de méchant d’u...,parse,Publié par wikistrike.com\n ...,[],NaN,NaN,2022-08-23T00:00:00,NaN,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,www.wikistrike.com,Le Forum économique mondial estime qu’il exist...
465,https://www.wikistrike.com/2022/08/le-masque-r...,"Le masque réduit les infections COVID de 0,06%",par Collectif Reinfocovid.\nCet article illust...,parse,Publié par wikistrike.com\n ...,[],NaN,NaN,2022-08-24T00:00:00,NaN,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,www.wikistrike.com,"Le masque réduit les infections COVID de 0,06%..."
466,https://www.wikistrike.com/2022/08/l-un-des-pl...,L’un des plus grands complexes mégalithiques d...,"En Espagne, le site d’une future plantation d’...",parse,Publié par wikistrike.com\n ...,[],NaN,NaN,2022-08-24T00:00:00,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,www.wikistrike.com,L’un des plus grands complexes mégalithiques d...


## Analyse du Corpus

In [38]:
import patat.ml.lex_analyser

In [39]:
lex = patat.ml.lex_analyser.LexAnalyser()

In [40]:
def merge_texts(texts):
    result = ''
    for text in texts:
        result = result + text + '\n'
    return result

In [41]:
def count_words_label(df,label):
    df_texts = pd.pivot_table(df, values='text', index=None, columns=label, aggfunc=merge_texts)
    wc = {}
    word_analysis = {}
    for key in df_texts.keys():
        text = df_texts[key]['text']
        count_colname = label+'_'+str(key)
        wc[key] = lex.count_tokens(lex.get_words(text))
        for word in wc[key]:
            word_dic = word_analysis.get(word,{})
            word_dic[count_colname]=wc[key][word]
            word_analysis[word]=word_dic
    return word_analysis

In [42]:
df_count = pd.DataFrame(count_words_label(df_corpus,'infox')).T

In [43]:
df_count = df_count.fillna(0)

In [46]:
df_count=df_count.sort_values(['infox_1.0','infox_0.0'],ascending=False)

In [47]:
df_count.to_csv('data/analysis/220930-ComptageRecueil.csv',decimal=',')